In [7]:
# Sent: ['Linux', 'is', 'the', 'best', 'OS']
# Labels: ['OS','IR','IR','IR','IR']
 
# Sent_s: ['Ubuntu', 'is', 'my', 'favorite', 'OS']
# Labels_l: ['OS','IR','IR','IR','IR']

In [147]:
data = [(['Linux', 'is', 'the', 'best', 'OS'], ['OS','IR','IR','IR','IR']),
(['Ubuntu', 'is', 'my', 'favourite', 'OS'], ['OS','IR','IR','IR','IR']),
       (['lokesh', 'is', 'my', 'name'], ['Person','IR','IR','IR']),
        (['lokesh', 'and', 'rajesh', 'are', 'my', 'friends'], ['Person','IR','Person','IR','IR','IR']),
       (['my', 'name', 'is', 'lokesh'], ['IR','IR','IR','Person'])]
corpus = []
for (doc, tags) in data:
    doc_tag = []
    for word, tag in zip(doc,tags):
        doc_tag.append((word, tag))
    corpus.append(doc_tag)
print(corpus)

[[('Linux', 'OS'), ('is', 'IR'), ('the', 'IR'), ('best', 'IR'), ('OS', 'IR')], [('Ubuntu', 'OS'), ('is', 'IR'), ('my', 'IR'), ('favourite', 'IR'), ('OS', 'IR')], [('lokesh', 'Person'), ('is', 'IR'), ('my', 'IR'), ('name', 'IR')], [('lokesh', 'Person'), ('and', 'IR'), ('rajesh', 'Person'), ('are', 'IR'), ('my', 'IR'), ('friends', 'IR')], [('my', 'IR'), ('name', 'IR'), ('is', 'IR'), ('lokesh', 'Person')]]


In [148]:
def doc2features(doc, i):
    word = doc[i][0]
#     print(i)
    # Features from current word
    features={
        'word.word': word,
    }
    # Features from previous word
    if i > 0:
        prevword = doc[i-1][0]
        features['word.prevword'] = prevword
    else:
        features['BOS'] = True # Special "Beginning of Sequence" tag
        
    # Features from next word
    if i < len(doc)-1:
        nextword = doc[i+1][0]
        features['word.nextword'] = nextword
    else:
        features['EOS'] = True # Special "End of Sequence" tag
    return features
 
def extract_features(doc):
    return [doc2features(doc, i) for i in range(len(doc))]
 
X = [extract_features(doc) for doc in corpus]
print(X)
# X = [[{'BOS': True, 'word.word': 'Linux', 'word.nextword': 'is'},
# {'word.nextword': 'the', 'word.word': 'is', 'word.prevword': 'Linux'},
# {'word.nextword': 'best', 'word.word': 'the', 'word.prevword': 'is'},
# {'word.nextword': 'OS', 'word.word': 'best', 'word.prevword': 'the'},
# {'EOS': True, 'word.word': 'OS', 'word.prevword': 'best'}],
# [{'BOS': True, 'word.word': 'Ubuntu', 'word.nextword': 'is'},
# {'word.nextword': 'my', 'word.word': 'is', 'word.prevword': 'Ubuntu'},
# {'word.nextword': 'favorite', 'word.word': 'my', 'word.prevword': 'is'},
# {'word.nextword': 'OS', 'word.word': 'favorite', 'word.prevword': 'my'},
# {'EOS': True, 'word.word': 'OS', 'word.prevword': 'favourite'}]]

[[{'word.word': 'Linux', 'BOS': True, 'word.nextword': 'is'}, {'word.word': 'is', 'word.prevword': 'Linux', 'word.nextword': 'the'}, {'word.word': 'the', 'word.prevword': 'is', 'word.nextword': 'best'}, {'word.word': 'best', 'word.prevword': 'the', 'word.nextword': 'OS'}, {'word.word': 'OS', 'word.prevword': 'best', 'EOS': True}], [{'word.word': 'Ubuntu', 'BOS': True, 'word.nextword': 'is'}, {'word.word': 'is', 'word.prevword': 'Ubuntu', 'word.nextword': 'my'}, {'word.word': 'my', 'word.prevword': 'is', 'word.nextword': 'favourite'}, {'word.word': 'favourite', 'word.prevword': 'my', 'word.nextword': 'OS'}, {'word.word': 'OS', 'word.prevword': 'favourite', 'EOS': True}], [{'word.word': 'lokesh', 'BOS': True, 'word.nextword': 'is'}, {'word.word': 'is', 'word.prevword': 'lokesh', 'word.nextword': 'my'}, {'word.word': 'my', 'word.prevword': 'is', 'word.nextword': 'name'}, {'word.word': 'name', 'word.prevword': 'my', 'EOS': True}], [{'word.word': 'lokesh', 'BOS': True, 'word.nextword': 'and

In [149]:
def get_labels(doc):
    return [tag for (token,tag) in doc]
y = [get_labels(doc) for doc in corpus]
print(y)

[['OS', 'IR', 'IR', 'IR', 'IR'], ['OS', 'IR', 'IR', 'IR', 'IR'], ['Person', 'IR', 'IR', 'IR'], ['Person', 'IR', 'Person', 'IR', 'IR', 'IR'], ['IR', 'IR', 'IR', 'Person']]


In [155]:
from sklearn_crfsuite.estimator import CRF
crf = CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=False,
)
crf.fit(X, y);

In [151]:
crf

CRF(algorithm='lbfgs', all_possible_states=None,
  all_possible_transitions=False, averaging=None, c=None, c1=0.1, c2=0.1,
  calibration_candidates=None, calibration_eta=None,
  calibration_max_trials=None, calibration_rate=None,
  calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
  gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=20,
  max_linesearch=None, min_freq=None, model_filename=None,
  num_memories=None, pa_type=None, period=None, trainer_cls=None,
  variance=None, verbose=False)

In [159]:
test = [['rajesh', 'is', 'my', 'name']]
X_test = extract_features(test)
print(crf.predict_marginals_single(X_test))

[{'OS': 0.29201078186845686, 'IR': 0.14427271076355122, 'Person': 0.5637165073679918}]


In [153]:
help(crf)

Help on CRF in module sklearn_crfsuite.estimator object:

class CRF(sklearn.base.BaseEstimator)
 |  python-crfsuite wrapper with interface siimlar to scikit-learn.
 |  It allows to use a familiar fit/predict interface and scikit-learn
 |  model selection utilities (cross-validation, hyperparameter optimization).
 |  
 |  Unlike pycrfsuite.Trainer / pycrfsuite.Tagger this object is picklable;
 |  on-disk files are managed automatically.
 |  
 |  Parameters
 |  ----------
 |  algorithm : str, optional (default='lbfgs')
 |      Training algorithm. Allowed values:
 |  
 |      * ``'lbfgs'`` - Gradient descent using the L-BFGS method
 |      * ``'l2sgd'`` - Stochastic Gradient Descent with L2 regularization term
 |      * ``'ap'`` - Averaged Perceptron
 |      * ``'pa'`` - Passive Aggressive (PA)
 |      * ``'arow'`` - Adaptive Regularization Of Weight Vector (AROW)
 |  
 |  min_freq : float, optional (default=0)
 |      Cut-off threshold for occurrence
 |      frequency of a feature. CRFsu

In [154]:
crf.classes_

['OS', 'IR', 'Person']

In [145]:
# # from nltk.tag.stanford import NERTagger
# import nltk.tag.stanford as st
# import os
# java_path = "/Java/jdk1.8.0_45/bin/java.exe"
# os.environ['JAVAHOME'] = java_path
# st = st.StanfordNERTagger('../ner-model.ser.gz','../stanford-ner.jar')
# tagging = st.tag(text.split()) 

In [146]:
from nltk.corpus import conll2000
import random
 
conll_data = list(conll2000.chunked_sents())
random.shuffle(conll_data)
train_sents = conll_data[:int(len(conll_data) * 0.8)]
test_sents = conll_data[int(len(conll_data) * 0.8 + 1):]